# Lançamento de projéteis: uma introdução

* PET - Física UFRN
* Petiano: Moisés da Rocha Alves
* Data: 14 de Setembro de 2022

O objetivo deste `Notebook` é introduzirmos o lançamento de projéteis, como o nome já indica. Descreveremos o problema e, a medida que formos prosseguindo, adicionaremos mais complexidade nele.

Dito isto, este `Notebook` estará dividido em três seções principais:
1. A descrição do lançamento de um projétil
2. A resistência do ar
3. Adicionando mais um grau de complexidade: a densidade do ar

Agora vamos as bibliotecas e ao `Notebook` de fato.

### Importando as bibliotecas necessárias

Neste `Notebook`, utilizaremos as bibliotecas `matplotlib`, `numpy` e `scipy`.

In [1]:
import matplotlib.pyplot as plt
import scipy.constants as sconst
import numpy as np

### Detalhes das bibliotecas

In [2]:
%load_ext version_information
%version_information Matplotlib, Numpy, Scipy

Software versions
Python 3.9.12 64bit [MSC v.1916 64 bit (AMD64)]
IPython 8.2.0
OS Windows 10 10.0.19042 SP0
Matplotlib 3.5.1
Numpy 1.21.5
Scipy 1.7.3
Mon Sep 12 19:38:49 2022 Hora Padrão de Buenos Aires

## 1. A descrição do lançamento de um projétil

Bom, primeiramente nos perguntemos: qual a maneira mais simples que temos de descrevermos um projétil? Queremos imaginar um projétil que seja disparado de algum lugar, com certa velocidade e ângulos específicos dados no início do lançamento.

Para responder a pergunta, faremos o que sempre é feito na física como um todo: abstrair o problema e tentar examinar-lo sem muitos graus de complexidade. Isto será feito para que possamos obter uma descrição mais fundamental (e também mais simples) do problema. Sendo assim, partiremos pra descrição física que o leitor já deve estar habituado: a de um lançamento oblíquo (não entrarei em detalhes neste `Notebook` sobre esta descrição, já que ela pode ser encontrada em vários lugares da internet).

Sendo assim, podemos descrever o projétil em sua forma mais fundamental com as seguintes equações diferenciais:

$$
\frac{d^2x}{dt^2} = 0
$$
$$
\frac{d^2y}{dt^2} = -g
$$

E, bom, essas equações diferenciais são simples 